<a href="https://colab.research.google.com/github/fernanda-palacios/ai-code-notebooks/blob/main/f_convolutional_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolutional Neural Networks for Digit Classification**

In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F


torch.manual_seed(64)
MNIST_preprocess = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize(0.1307, 0.3081)])

MNIST_train = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform = MNIST_preprocess)

MNIST_test = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform = MNIST_preprocess)

train_loader = torch.utils.data.DataLoader(
    MNIST_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(MNIST_test, 64)

In [ ]:
class MNISTConv(nn.Module):
    def __init__(self):
        super(MNISTConv, self).__init__()
        self.conv_1 = nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_2 = nn.Conv2d(6, 2, 5)
        self.linear = nn.Linear(50, 10)

    def forward(self, x):
        a = F.relu(self.conv_1(x))
        a = self.pool(a)
        a = F.relu(self.conv_2(a))
        a = self.pool(a)
        return self.linear(torch.flatten(a, 1))

conv_model = MNISTConv()

In [ ]:
def num_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of trainable parameters: ", num_trainable_params(conv_model))

Number of trainable parameters:  872


## Training



In [ ]:
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tqdm


def train(dataset, loader, model, criterion, optimizer, epochs=10):

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Device:", device)
    model = model.to(device)
    model.train()

    for epoch in range(epochs):
        full_loss = 0.0
        n_correct = 0

        for images, labels in tqdm(loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            full_loss += loss.item() * len(labels)
            n_correct += (predicted == labels.data).sum()
        average_loss = full_loss / len(dataset)
        accuracy = n_correct / len(dataset) * 100

        print(f"Epoch {epoch+1} training average loss: {average_loss:.3f}",
              f"with {accuracy:.2f}% accuracy")


def test(dataset, loader, model):
    model.eval()
    n_correct = 0
    with torch.no_grad():
        for images, labels in tqdm(loader):
            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            n_correct += (predicted == labels.data).sum()

    print(f"Test accuracy: {n_correct/len(dataset)}")

In [ ]:
criterion = nn.CrossEntropyLoss()
conv_optimizer = torch.optim.SGD(
    conv_model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)

train(MNIST_train, train_loader, conv_model, criterion, conv_optimizer)
test(MNIST_test, test_loader, conv_model)

Device: cpu


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1 training average loss: 0.703 with 76.46% accuracy


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch 2 training average loss: 0.226 with 93.28% accuracy


...

Epoch 9 training average loss: 0.151 with 95.61% accuracy


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch 10 training average loss: 0.149 with 95.54% accuracy


  0%|          | 0/157 [00:00<?, ?it/s]

Test accuracy: 0.9592000246047974
